In [ ]:
from sweat4science.workspace.Workspace import Workspace
from sweat4science.evaluation.sessionset import MF_sessionset as mfs
import sweat4science as s4s

import os
import numpy as np

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json

from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import KFold
from s4s_rnn import utils

# import sys
# print(sys.path)

In [ ]:
workspace_folder = "/home/minh/workspace/git/rnd/session-data"
ws = Workspace(workspace_folder)
sessions = mfs.ICT_indoor(ws)


In [ ]:
import re

for session in sessions:
    if len(re.findall("slope", str(session))) > 0:
        sessions.remove(session)
    pass

sessions = np.array(sessions)
print("\n".join(map(str, sessions)))


In [ ]:
def evaluate_cross_validation(sessions, result_dir, experiment_name, date_string,
                              time_steps, hidden_neurons, num_epoch,
                              input_dim=4, save_plot=False):
    # Construct base name
    base_name = experiment_name + "_" + str(time_steps) + "step_" + str(input_dim) + "in_" \
        + str(hidden_neurons) + "hidden_" + "20161114" + "_" + str(num_epoch) + "epoch_"
    base_name = os.path.join(result_dir, base_name)
    print("\nBase name: %s\n" % (base_name))

    # Open model
    model_file_name = base_name  + "model.json"
    json_file = open(model_file_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    # Cross validation testing
    test_predictions = None
    test_actual_outputs = None
    kf = KFold(len(sessions))
    for train_index, test_index in kf.split(sessions):
        test_sessions = sessions[test_index]
        print("\nTesting on:\n" + "\n".join(map(str, test_sessions)))

        test_data_x, test_data_y, test_mean, test_std = \
            utils.get_data_from_sessions(test_sessions, time_steps, return_norm=True)

        loaded_model = model_from_json(loaded_model_json)

        match = re.match('.+/running_indoor_(.+)/(\d+)>', str(test_sessions[0]))
        cross_validation_name = base_name + match.groups()[0] + "_" + match.groups()[1] + "_"

        test_data_y_unnormed, test_predict_unnormed, mse = \
            utils.evaluate_model(loaded_model, cross_validation_name + "weights.h5",
                                 test_data_x, test_data_y, test_mean[-1], test_std[-1])

        print("MSE: %.5f\nRMSE: %.5f" % (mse, np.sqrt(mse)))

        utils.plot_predictions(test_predict_unnormed, test_data_y_unnormed, cross_validation_name + "result.png",
                               "Heart rate simulation for running_indoor_" + match.groups()[0]
                               + "/" + match.groups()[1] + ", lookback of " + str(time_steps))

        test_predictions = test_predict_unnormed if test_predictions is None \
            else np.append(test_predictions, test_predict_unnormed, axis=0)
        test_actual_outputs = test_data_y_unnormed if test_actual_outputs is None \
            else np.append(test_actual_outputs, test_data_y_unnormed, axis=0)

        pass

    mse = np.mean((test_predictions - test_actual_outputs)**2)
    rmse = np.sqrt(mse)
    print("\nOverall results:\n MSE: %.5f\n RMSE: %.5f" % (mse, rmse))
    return


In [ ]:
evaluate_cross_validation(sessions=sessions, result_dir="train_results", experiment_name="lstm_indoor",
                          date_string="20161114", time_steps=5, hidden_neurons=400, num_epoch=150)

In [ ]:
data = np.genfromtxt("artificial_data/sinx_plus_x.csv", delimiter=',')
num_train = int(0.8*len(data))
train_data_x_ = data[:num_train, :-1]
train_data_y_ = data[:num_train, -1:]
test_data_x_ = data[num_train:, :-1]
test_data_y_ = data[num_train:, -1:]

plt.figure(figsize=(10, 7))
plt.plot(train_data_y_, 'g*')
plt.show()

input_dim = 2
output_dim = 1
hidden_neurons = 400
num_epoch = 20
for ntsteps in [5, 10, 15]:
    base_name = "lstm_sinx_plus_x_" + str(ntsteps) + "step_" + str(input_dim) +\
                "in_" + str(hidden_neurons) + "hidden_" + "20161108" +\
                "_" + str(num_epoch) + "epoch_"
    base_name = os.path.join("train_results", base_name)
    model_file_name = base_name  + "model.json"
    json_file = open(model_file_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)

    test_data_x = utils.reshape_array_by_time_steps(test_data_x_, time_steps=ntsteps)
    test_data_y = test_data_y_[-len(test_data_x):]

    loaded_model.load_weights(base_name + "weights.h5")
    loaded_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])

    test_predict = loaded_model.predict(test_data_x)
    mse = np.mean((test_predict - test_data_y)**2)
    rmse = np.sqrt(mse)

    print("MSE: %.5f\nRMSE: %.5f" % (mse, rmse) )

    # Plot
    plt.figure(figsize=(10, 7))
    plt.plot(test_predict, 'r+')
    plt.plot(test_data_y, 'g*')
    plt.show()
    pass

